In [1]:
import time
import grpc
import chunk_pb2  # Import your generated protobuf file
import chunk_pb2_grpc  # Import your generated gRPC file
import numpy as np

def transfer_grpc(file_size, num_runs, source_url, destination_url):
    transfer_times = []
    throughput_values = []
    total_bytes_transferred = 0

    for _ in range(num_runs):
        start_time = time.time()
        with grpc.insecure_channel(source_url) as channel:
            stub = chunk_pb2_grpc.FileServerStub(channel)
            print("stub = ",stub)
            #response = stub.TransferFile(chunk_pb2.TransferRequest(file_size=file_size))
            #response = stub.download(chunk_pb2.download(file_size=file_size))
            #response = stub.TransferFile(chunk_pb2.TransferRequest(file_size=file_size))
            #response = stub.download(chunk_pb2_grpc.TransferRequest(file_size=file_size))
            response = stub.download(chunk_pb2.Request(name=destination_url))

            with open(destination_url, 'wb') as f:
                #f.write(response.file_content)
                for chunk in response:
                    f.write(chunk.buffer)
            end_time = time.time()
            transfer_time = end_time - start_time
            transfer_times.append(transfer_time)
            total_bytes_transferred += file_size
            throughput = file_size / transfer_time
            throughput_values.append(throughput)
            print("Ok")
    return transfer_times, throughput_values, total_bytes_transferred

#source_url_grpc = '127.0.0.1:50054/link.txt'  # Adjust server IP and port if needed
#source_url_grpc = '192.168.1.11:50054/link.txt'  # Adjust server IP and port if needed
#source_url_grpc = 'localhost:50055/link.txt'  # Adjust server IP and port if needed
source_url_grpc = '172.17.102.16:50055/link.txt'  # Adjust server IP and port if needed

destination_url_grpc = 'link2.txt'  # Save in local directory
file_size = 10 * 1024  # 10KB
num_runs = 10

transfer_times_grpc, throughput_values, total_bytes_transferred = transfer_grpc(file_size, num_runs, source_url_grpc, destination_url_grpc)

average_transfer_time = np.mean(transfer_times_grpc)
std_dev_transfer_time = np.std(transfer_times_grpc)

total_transfer_time = sum(transfer_times_grpc)
average_throughput = total_bytes_transferred / total_transfer_time
std_dev_throughput = np.std(throughput_values)

print("Average Transfer Time:", average_transfer_time)
print("Standard Deviation of Transfer Time:", std_dev_transfer_time)
print("Average Throughput:", average_throughput, "bytes per second")
print("Standard Deviation of Throughput:", std_dev_throughput, "bytes per second")


stub =  <chunk_pb2_grpc.FileServerStub object at 0x00000205BF350520>


_MultiThreadedRendezvous: <_MultiThreadedRendezvous of RPC that terminated with:
	status = StatusCode.UNAVAILABLE
	details = "DNS resolution failed for 172.17.102.16:50055/link.txt: UNAVAILABLE: WSA Error"
	debug_error_string = "UNKNOWN:Error received from peer  {created_time:"2024-02-29T21:39:29.4349393+00:00", grpc_status:14, grpc_message:"DNS resolution failed for 172.17.102.16:50055/link.txt: UNAVAILABLE: WSA Error"}"
>